In [1]:
from shutil import copyfile
from PIL import Image
import os
import random
import pandas as pd
import cv2

In [2]:
images_dir = "yolo_signs_images/"
labels_dir = "yolo_signs_labels/"
output_dir = "traffic_signs/"

classes = [
    "CrossWalk",
    "NoPass",
    "Park",
    "RoadWork",
]

In [3]:
# write class names
with open(output_dir + "class.names", "w") as f:
    for name in classes:
        f.write(name + "\n")

# write data file        
with open(output_dir + "/traffic_signs.data", "w") as f:
    f.write("classes = " + str(len(classes)) + "\n")
    f.write("train = /home/lab/workspace/darknet/data/traffic_signs/train.txt" + "\n")
    f.write("valid = /home/lab/workspace/darknet/data/traffic_signs/test.txt" + "\n")
    f.write("names = /home/lab/workspace/darknet/data/traffic_signs/class.names" + "\n")
    f.write("backup = /home/lab/workspace/darknet/backup/" + "\n")

In [4]:
df = pd.DataFrame(columns=['Path', 'ClassId', 'Roi.X1', 'Roi.Y1', 'Roi.X2', 'Roi.Y2'])

for root, dirs, files in os.walk(labels_dir):
        for file in files:
            label_path = os.path.join(root,file)
            image_path = label_path.replace(labels_dir, images_dir)
            
            #print(label_path)
            
            lbl_str = label_path.split("/")[1]
            lbl_id = classes.index(lbl_str)
            
            nmbr = os.path.basename(image_path).split('.')[0]
            with open(label_path, "r") as f:
                lines = f.readlines()
                if len(lines) > 1:
                    tmp = lines[1].split(" ")
                    df = df.append({'Path': nmbr, 'ClassId': lbl_id, 'Roi.X1': int(tmp[0]), 'Roi.Y1': int(tmp[1]), 'Roi.X2': int(tmp[2]), 'Roi.Y2': int(tmp[3])}, ignore_index=True)
            
print(len(df))
print(df[0:5])

3866
    Path ClassId Roi.X1 Roi.Y1 Roi.X2 Roi.Y2
0  05915       1    636    195    715    273
1  02374       1    738    146    850    269
2  05142       1    806     68    970    211
3  00176       1     68    225    162    301
4  05741       1    761    157    853    254


In [5]:
# preprocess training dataset
for i, index in enumerate(df.index):
    c = df.ClassId[index]
    n = df.Path[index]
    w = 1280
    h = 640
    x1 = df["Roi.X1"][index]
    y1 = df["Roi.Y1"][index]
    x2 = df["Roi.X2"][index]
    y2 = df["Roi.Y2"][index]
    
    ox = ((x1 + x2)/2)/w
    oy = ((y1 + y2)/2)/h
    ow = abs(x1 - x2)/w
    oh = abs(y1 - y2)/h
    
    string = str(c) + " " + str(ox) + " " + str(oy) + " " + str(ow) + " " + str(oh)
    
    print(n)
    print(string)
    with open(output_dir + n + ".txt", "a") as f:
        f.write(string + "\n")
        
random.seed(123)
file_history = set()
with open(output_dir + "train.txt", "w") as f_train, open(output_dir + "test.txt", "w") as f_test:
    for i, index in enumerate(df.index):
        n = df.Path[index]
        if n not in file_history:
            file_history.add(n)
            #copyfile(images_dir + n + ".jpg", output_dir + n + ".jpg")
            n = "data/traffic_signs/" + n + ".JPEG"
            rnd = random.randint(0,4) #Training 75%, Validation 25%
            if rnd == 0:
                f_test.write(n + "\n")
            else:
                f_train.write(n + "\n")

05915
1 0.527734375 0.365625 0.06171875 0.121875
02374
1 0.6203125 0.32421875 0.0875 0.1921875
05142
1 0.69375 0.21796875 0.128125 0.2234375
00176
1 0.08984375 0.4109375 0.0734375 0.11875
05741
1 0.63046875 0.32109375 0.071875 0.1515625
01590
1 0.821484375 0.15703125 0.26640625 0.3078125
05892
1 0.0390625 0.40390625 0.075 0.1296875
06617
1 0.671875 0.29375 0.11875 0.228125
06601
1 0.571875 0.346875 0.078125 0.171875
05749
1 0.63515625 0.30078125 0.0828125 0.1734375
04509
1 0.621875 0.2546875 0.1015625 0.20625
03045
1 0.79609375 0.18359375 0.1859375 0.3515625
05758
1 0.64375 0.2765625 0.0953125 0.196875
02993
1 0.3296875 0.39453125 0.0671875 0.1203125
00928
1 0.7484375 0.16328125 0.2453125 0.3015625
06566
1 0.3109375 0.40234375 0.0578125 0.1140625
02359
1 0.502734375 0.35703125 0.07265625 0.1515625
04463
1 0.233203125 0.3671875 0.06484375 0.1125
07187
1 0.076953125 0.41875 0.08515625 0.128125
05932
1 0.623046875 0.34140625 0.08515625 0.1828125
04525
1 0.7078125 0.16484375 0.146875 0.304

05924
1 0.61953125 0.35859375 0.075 0.1640625
07249
1 0.693359375 0.25625 0.12734375 0.275
05897
1 0.155078125 0.4046875 0.07109375 0.13125
06604
1 0.597265625 0.3359375 0.07578125 0.18125
00212
1 0.550390625 0.346875 0.07421875 0.15
02334
1 0.1890625 0.409375 0.071875 0.1375
05755
1 0.642578125 0.28828125 0.09296875 0.1734375
02988
1 0.22109375 0.39921875 0.071875 0.1265625
03660
1 0.80859375 0.19375 0.1859375 0.365625
05737
1 0.56640625 0.33671875 0.0765625 0.1390625
03597
1 0.101953125 0.40859375 0.07421875 0.1421875
05111
1 0.5859375 0.32890625 0.071875 0.1421875
01571
1 0.628125 0.28828125 0.1078125 0.2109375
05721
1 0.048046875 0.3765625 0.08515625 0.13125
00198
1 0.51484375 0.36953125 0.0625 0.1359375
05159
1 0.83828125 0.1359375 0.203125 0.26875
00249
1 0.66953125 0.20078125 0.16875 0.3390625
05902
1 0.2796875 0.39765625 0.0625 0.1265625
06623
1 0.69609375 0.23828125 0.153125 0.3046875
07247
1 0.6921875 0.27109375 0.1296875 0.2640625
05960
1 0.713671875 0.2375 0.12578125 0.3
06

02394
1 0.691796875 0.25546875 0.14453125 0.2921875
04491
1 0.571484375 0.31640625 0.07265625 0.1515625
04533
1 0.748046875 0.13984375 0.19296875 0.2671875
04474
1 0.44921875 0.353125 0.0625 0.125
04530
1 0.73125 0.15078125 0.175 0.2890625
04519
1 0.656640625 0.21328125 0.12890625 0.2609375
00875
1 0.5515625 0.353125 0.071875 0.146875
07200
1 0.34375 0.4 0.05625 0.115625
00199
1 0.514453125 0.36953125 0.06328125 0.1421875
00217
1 0.56484375 0.3390625 0.0859375 0.165625
01538
1 0.4875 0.3765625 0.059375 0.125
03017
1 0.65703125 0.32421875 0.0890625 0.2015625
07237
1 0.670703125 0.31796875 0.09921875 0.2296875
02375
1 0.62265625 0.321875 0.0984375 0.196875
01583
1 0.755859375 0.18203125 0.20390625 0.3421875
00906
1 0.651171875 0.2796875 0.12421875 0.253125
02986
1 0.180859375 0.40546875 0.07734375 0.1234375
03666
1 0.847265625 0.17265625 0.22421875 0.3359375
06602
1 0.594140625 0.33515625 0.08203125 0.1578125
07225
1 0.502734375 0.3546875 0.07265625 0.153125
02338
1 0.275390625 0.4007812

05781
1 0.751953125 0.1609375 0.16015625 0.30625
04511
1 0.621484375 0.25390625 0.10390625 0.2140625
06561
1 0.2109375 0.41796875 0.06875 0.1203125
06445
3 0.601171875 0.32578125 0.08203125 0.1671875
04967
3 0.65078125 0.3203125 0.103125 0.196875
02834
3 0.20625 0.41171875 0.075 0.1171875
02224
3 0.606640625 0.3171875 0.09921875 0.19375
05590
3 0.621875 0.325 0.0953125 0.1875
00689
3 0.54921875 0.37109375 0.06875 0.1078125
00106
3 0.7796875 0.1671875 0.203125 0.3125
04301
3 0.47734375 0.37578125 0.065625 0.1328125
07082
3 0.651171875 0.28125 0.10390625 0.21875
07054
3 0.325 0.38203125 0.06875 0.1234375
06439
3 0.583203125 0.3359375 0.07734375 0.153125
02867
3 0.614453125 0.34296875 0.09140625 0.1703125
00709
3 0.670703125 0.325 0.08984375 0.1625
00720
3 0.605859375 0.3140625 0.09921875 0.18125
06458
3 0.644140625 0.28984375 0.09921875 0.2046875
00053
3 0.58671875 0.359375 0.06875 0.13125
00089
3 0.691015625 0.278125 0.11484375 0.221875
07106
3 0.69609375 0.1640625 0.1765625 0.315625
02

07683
3 0.549609375 0.334375 0.07578125 0.15
07047
3 0.191796875 0.38515625 0.07734375 0.1265625
00094
3 0.70625 0.25 0.1453125 0.259375
00703
3 0.671484375 0.34296875 0.07265625 0.1453125
02896
3 0.669140625 0.22109375 0.16640625 0.3140625
06425
3 0.45859375 0.371875 0.0625 0.121875
04998
3 0.8640625 0.15390625 0.271875 0.2984375
04344
3 0.681640625 0.259375 0.14609375 0.296875
05558
3 0.1265625 0.41875 0.08125 0.1375
02886
3 0.64296875 0.28515625 0.1234375 0.2484375
02893
3 0.662109375 0.240625 0.15546875 0.284375
04963
3 0.642578125 0.3296875 0.09921875 0.1875
04300
3 0.46640625 0.378125 0.0671875 0.13125
04975
3 0.66640625 0.28984375 0.1203125 0.2328125
00006
3 0.48046875 0.446875 0.04375 0.078125
02245
3 0.673828125 0.20859375 0.17734375 0.3484375
04325
3 0.610546875 0.328125 0.09765625 0.190625
02833
3 0.190234375 0.409375 0.06328125 0.134375
00751
3 0.7828125 0.14765625 0.2484375 0.2921875
04312
3 0.575 0.35390625 0.0765625 0.1578125
04980
3 0.70390625 0.265625 0.140625 0.271875

...

07567
0 0.891015625 0.15625 0.21953125 0.290625
04119
0 0.03203125 0.42265625 0.0625 0.1171875
03316
0 0.664453125 0.35703125 0.06796875 0.1296875
06273
0 0.647265625 0.334375 0.08515625 0.178125
02058
0 0.6390625 0.30078125 0.1078125 0.2078125
03341
0 0.68828125 0.3109375 0.1 0.184375
02685
0 0.538671875 0.375 0.05859375 0.109375
03296
0 0.310546875 0.40625 0.06015625 0.103125
04156
0 0.633203125 0.359375 0.06484375 0.1375
02662
0 0.069921875 0.421875 0.07109375 0.109375
01227
0 0.6125 0.3359375 0.06875 0.1375
04824
0 0.6359375 0.3109375 0.0890625 0.1875
01998
0 0.162109375 0.41328125 0.06171875 0.1046875
04194
0 0.661328125 0.215625 0.13828125 0.2875
04160
0 0.62734375 0.34609375 0.0703125 0.1453125
06919
0 0.6375 0.30859375 0.0921875 0.1890625
01881
0 0.704296875 0.290625 0.09453125 0.178125
02067
0 0.703125 0.25 0.1375 0.2625
00562
0 0.58125 0.32734375 0.0734375 0.1515625
07544
0 0.720703125 0.29140625 0.10859375 0.2140625
02017
0 0.494921875 0.39140625 0.05078125 0.1109375
02082
0

06229
0 0.080078125 0.42109375 0.08046875 0.1203125
02050
0 0.628515625 0.334375 0.08984375 0.171875
02669
0 0.22890625 0.4109375 0.059375 0.1
06255
0 0.5953125 0.37109375 0.0609375 0.1328125
07499
0 0.16953125 0.41484375 0.0703125 0.1078125
03318
0 0.66953125 0.35390625 0.0703125 0.1359375
07515
0 0.4859375 0.390625 0.0578125 0.115625
02687
0 0.554296875 0.3765625 0.05703125 0.115625
04161
0 0.6265625 0.34609375 0.06875 0.1453125
00533
0 0.347265625 0.38671875 0.05390625 0.1078125
06949
0 0.889453125 0.1328125 0.22265625 0.25625
01205
0 0.3484375 0.37890625 0.0515625 0.1046875
01865
0 0.569921875 0.3375 0.06484375 0.134375
02002
0 0.25078125 0.41171875 0.0578125 0.1078125
07557
0 0.784375 0.2109375 0.1484375 0.290625
02713
0 0.678125 0.33359375 0.0890625 0.1765625
02718
0 0.68671875 0.31796875 0.1 0.1859375
05416
0 0.32578125 0.4046875 0.059375 0.109375
04785
0 0.060546875 0.42109375 0.06484375 0.1109375
04195
0 0.669921875 0.20703125 0.14453125 0.2984375
06882
0 0.2453125 0.409375 0.

00394
2 0.678125 0.3 0.1015625 0.215625
06137
2 0.67421875 0.19453125 0.1390625 0.3015625
01046
2 0.54296875 0.353125 0.0671875 0.14375
05298
2 0.692578125 0.26640625 0.10078125 0.2109375
03809
2 0.65703125 0.29140625 0.078125 0.1578125
01727
2 0.763671875 0.25390625 0.12578125 0.2578125
03987
2 0.71796875 0.321875 0.0875 0.16875
01061
2 0.679296875 0.30234375 0.09765625 0.1984375
03149
2 0.2265625 0.36953125 0.0625 0.1046875
02521
2 0.448046875 0.3625 0.05390625 0.1125
06793
2 0.71953125 0.1859375 0.1625 0.340625
05315
2 0.790625 0.1609375 0.203125 0.30625
06765
2 0.676953125 0.325 0.08046875 0.178125
05285
2 0.663671875 0.31953125 0.08671875 0.1734375
04689
2 0.78984375 0.1640625 0.1703125 0.321875
03175
2 0.586328125 0.31640625 0.06796875 0.1390625
06093
2 0.5484375 0.35703125 0.0640625 0.1359375
03826
2 0.65625 0.22109375 0.1109375 0.2328125
06127
2 0.705859375 0.259375 0.12265625 0.25625
00371
2 0.57421875 0.36015625 0.0671875 0.1453125
03994
2 0.7125 0.31484375 0.090625 0.1859375

05318
2 0.8234375 0.1484375 0.228125 0.2875
06116
2 0.614453125 0.29453125 0.08203125 0.1890625
00391
2 0.6734375 0.3078125 0.0875 0.190625
04637
2 0.48828125 0.384375 0.046875 0.1
04625
2 0.16640625 0.40546875 0.0578125 0.1140625
03951
2 0.2640625 0.3921875 0.0578125 0.10625
06126
2 0.703125 0.25703125 0.1125 0.2453125
03959
2 0.418359375 0.38125 0.05078125 0.103125
05305
2 0.726171875 0.215625 0.14609375 0.28125
03148
2 0.194140625 0.3703125 0.07109375 0.10625
04686
2 0.770703125 0.1828125 0.16484375 0.31875
01062
2 0.68125 0.30078125 0.0953125 0.2109375
05253
2 0.2515625 0.409375 0.0484375 0.10625
00405
2 0.7828125 0.23671875 0.1375 0.2953125
03808
2 0.65703125 0.2953125 0.0765625 0.165625
03803
2 0.651171875 0.29921875 0.07265625 0.1484375
01044
2 0.541015625 0.34921875 0.06640625 0.1609375
04003
2 0.645703125 0.29140625 0.10390625 0.2109375
06107
2 0.594140625 0.3328125 0.07890625 0.165625
01679
2 0.35546875 0.39453125 0.06875 0.1234375
03782
2 0.340625 0.36328125 0.05625 0.101562

06747
2 0.558203125 0.359375 0.06328125 0.13125
01668
2 0.05 0.41484375 0.075 0.1265625
00413
2 0.843359375 0.17109375 0.14765625 0.3328125
02535
2 0.5234375 0.33046875 0.0609375 0.1171875
02540
2 0.5375 0.3171875 0.065625 0.134375
04648
2 0.65546875 0.34375 0.0765625 0.15
03171
2 0.5859375 0.32578125 0.0609375 0.1140625
06759
2 0.687890625 0.32578125 0.08203125 0.1546875
07410
2 0.7453125 0.2109375 0.1515625 0.3125
03783
2 0.366015625 0.3625 0.05234375 0.10625
04635
2 0.372265625 0.3984375 0.05703125 0.10625
06787
2 0.6984375 0.23828125 0.134375 0.2828125
07381
2 0.635546875 0.32734375 0.06171875 0.1390625
06084
2 0.389453125 0.375 0.05703125 0.125
00402
2 0.71171875 0.25234375 0.115625 0.2546875
05282
2 0.66171875 0.328125 0.0890625 0.16875
07376
2 0.571484375 0.3515625 0.07109375 0.15
03799
2 0.64453125 0.3171875 0.0640625 0.134375
03142
2 0.0484375 0.371875 0.0765625 0.11875
04015
2 0.671484375 0.22265625 0.13984375 0.2859375
06073
2 0.149609375 0.4015625 0.07265625 0.11875
00411
2